# SANS reduction at RID

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import matplotlib.colors as mcolors
from scipy.optimize import curve_fit
from math import pi as pi
import re

## 1. File processing and basic plotting

In [1]:
from sansdata import *

In [ ]:
glassyC_sample = SansData('data/memb_BS_Q1_6_0Ang.mpa')
glassyC_sample.plot_integrated_intensity(axis=0, title='Integrated Intensity over X-axis')
glassyC_sample.plot_2d()

## 2. Automatic beam stop detection using OpenCV
The process is as follows:
- A threshold is applied to the image
- Contours are detected
- From these contours, a single minimum area rectangle is fitted

In [ ]:
import cv2
# Convert Pillow Image to NumPy array
img_np = np.array(glassyC_sample.intensity)
print(img_np.max())
print(img_np.min())
img = (np.round(img_np / img_np.max() * 255)).astype(np.uint8)

ret, thresh = cv2.threshold(img, 1, 10, cv2.THRESH_BINARY)
plt.imshow(thresh)

In [ ]:
# Detects edges
from numpy import copy


edged = cv2.Canny(thresh, 2, 20, apertureSize=3, L2gradient=True)
plt.imshow(edged)

In [ ]:


# Find contours in the binary image
contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
edge_copy = copy(edged)
cv2.drawContours(edge_copy, contours, -1, (255, 255, 255), 3) 
plt.imshow(edge_copy)